In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lower, regexp_replace, when
from pyspark.ml.feature import Tokenizer, StopWordsRemover

import random
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

from tensorflow.keras.preprocessing.text import Tokenizer as KTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

spark = SparkSession.builder \
    .appName("AmazonSentimentBiLSTM") \
    .getOrCreate()


[nltk_data] Downloading package wordnet to /home/jaimin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jaimin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
def load_dataset(path):
    df = spark.read.csv(
        path,
        header=False,
        inferSchema=True
    )
    df = df.toDF("rating", "review_title", "review_text")

    df = df.withColumn(
        "sentiment",
        when(df.rating >= 3, "positive").otherwise("negative")
    )

    return df.select("review_text", "sentiment")


In [9]:
train_df = load_dataset("data/train.csv")
test_df  = load_dataset("data/test.csv")

train_df.show(3, truncate=False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------+
|review_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |sentiment|
+---------------------------------

In [10]:
from pyspark.sql.functions import col, lower, regexp_replace, trim

def spark_preprocess(df):
    # ✅ Replace NULL text with empty string
    df = df.fillna({"review_text": ""})

    # ✅ Clean text
    df = df.withColumn(
        "clean_text",
        trim(lower(regexp_replace(col("review_text"), "[^a-zA-Z\\s]", "")))
    )

    # ✅ Drop rows that are empty AFTER cleaning
    df = df.filter(col("clean_text") != "")

    # ✅ Tokenization (SAFE now)
    tokenizer = Tokenizer(
        inputCol="clean_text",
        outputCol="tokens"
    )
    df = tokenizer.transform(df)

    # ✅ Stopword removal
    remover = StopWordsRemover(
        inputCol="tokens",
        outputCol="filtered_tokens"
    )
    df = remover.transform(df)

    return df


In [11]:
train_df = spark_preprocess(train_df)
test_df  = spark_preprocess(test_df)


In [12]:
train_df, val_df = train_df.randomSplit([0.875, 0.125], seed=42)


In [7]:
train_df, val_df = split_train_val(train_df)


In [13]:
def spark_to_python(df, limit):
    df = df.limit(limit)   # 🔥 CRITICAL

    data = df.select("filtered_tokens", "sentiment") \
        .rdd.map(
            lambda x: (" ".join(x[0]), 1 if x[1] == "positive" else 0)
        ).collect()

    texts, labels = zip(*data)
    return list(texts), list(labels)


In [14]:
X_train, y_train = spark_to_python(train_df, limit=40000)
X_val, y_val     = spark_to_python(val_df,   limit=5000)
X_test, y_test   = spark_to_python(test_df,  limit=10000)

len(X_train), len(X_val), len(X_test)


(40000, 5000, 10000)

In [21]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

def vectorize_text(X_train, X_val, X_test, max_words=20000, max_len=200):
    tokenizer = Tokenizer(
        num_words=max_words,
        oov_token="<OOV>"
    )
    tokenizer.fit_on_texts(X_train)   # ❗ train data only

    def encode(texts):
        seq = tokenizer.texts_to_sequences(texts)
        return pad_sequences(seq, maxlen=max_len)

    return (
        encode(X_train),
        encode(X_val),
        encode(X_test),
        tokenizer
    )


In [22]:
X_train_seq, X_val_seq, X_test_seq, keras_tokenizer = vectorize_text(
    X_train, X_val, X_test
)


In [23]:
X_train_seq.shape


(40000, 200)

In [25]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout

def build_bilstm(vocab_size=20000, embed_dim=128, max_len=200):
    model = Sequential([
        Embedding(vocab_size, embed_dim, input_length=max_len),
        Bidirectional(LSTM(64, return_sequences=True)),
        Dropout(0.5),
        Bidirectional(LSTM(32)),
        Dense(1, activation="sigmoid")
    ])

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [26]:
import numpy as np

X_train_seq = np.array(X_train_seq)
X_val_seq   = np.array(X_val_seq)
X_test_seq  = np.array(X_test_seq)

y_train = np.array(y_train)
y_val   = np.array(y_val)
y_test  = np.array(y_test)


In [28]:
model = build_bilstm()
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_4 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_5 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
history = model.fit(
    X_train_seq, y_train,
    validation_data=(X_val_seq, y_val),
    epochs=5,
    batch_size=128
)


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 276ms/step - accuracy: 1.0000 - loss: 0.0083 - val_accuracy: 1.0000 - val_loss: 6.0584e-05
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 90s 286ms/step - accuracy: 1.0000 - loss: 4.3716e-05 - val_accuracy: 1.0000 - val_loss: 2.6600e-05
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 95s 303ms/step - accuracy: 1.0000 - loss: 2.1634e-05 - val_accuracy: 1.0000 - val_loss: 1.4421e-05
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 85s 293ms/step - accuracy: 1.0000 - loss: 1.2390e-05 - val_accuracy: 1.0000 - val_loss: 9.2556e-06
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 127s 406ms/step - accuracy: 1.0000 - loss: 8.4364e-06 - val_accuracy: 1.0000 - val_loss: 6.6360e-06


In [30]:
loss, acc = model.evaluate(X_test_seq, y_test)
print("Clean Test Accuracy:", acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 35ms/step - accuracy: 1.0000 - loss: 6.6355e-06
Clean Test Accuracy: 1.0


In [31]:
import random
from nltk.corpus import wordnet

# 1️⃣ Randomly remove 20% words
def remove_words(text, ratio=0.2):
    words = text.split()
    if len(words) < 2:
        return text
    keep = max(1, int(len(words) * (1 - ratio)))
    return " ".join(random.sample(words, keep))


# 2️⃣ Introduce 10% spelling mistakes
def spelling_noise(text, ratio=0.1):
    words = text.split()
    n = int(len(words) * ratio)

    for i in random.sample(range(len(words)), min(n, len(words))):
        if len(words[i]) > 2:
            pos = random.randint(0, len(words[i]) - 2)
            w = list(words[i])
            w[pos], w[pos+1] = w[pos+1], w[pos]
            words[i] = "".join(w)

    return " ".join(words)


# 3️⃣ Replace 20% words with synonyms
def synonym_replace(text, ratio=0.2):
    words = text.split()
    n = int(len(words) * ratio)

    for i in random.sample(range(len(words)), min(n, len(words))):
        syns = wordnet.synsets(words[i])
        if syns:
            words[i] = syns[0].lemmas()[0].name()

    return " ".join(words)


In [32]:
X_test_noisy = []

for text in X_test:
    t = remove_words(text, ratio=0.2)
    t = spelling_noise(t, ratio=0.1)
    t = synonym_replace(t, ratio=0.2)
    X_test_noisy.append(t)


In [34]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

X_test_noisy_seq = pad_sequences(
    keras_tokenizer.texts_to_sequences(X_test_noisy),
    maxlen=200
)

X_test_noisy_seq = np.array(X_test_noisy_seq)


In [35]:
loss_noisy, acc_noisy = model.evaluate(
    X_test_noisy_seq,
    np.array(y_test),
    verbose=1
)

print("Noisy Test Accuracy:", acc_noisy)


313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 1.0000 - loss: 6.6344e-06
Noisy Test Accuracy: 1.0
